In [1]:
import glob
from glob import glob
import pandas as pd
import numpy as np
import string

In [2]:
from os import listdir
import nltk
from os.path import isfile, join
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Data Cleaning

In [3]:
def find_features(document,vocab):
    words=(document)
    feature=[]
    for w in vocab:
        if(w in words):
            feature.append(int(words.count(w)))
        else:
            feature.append(int(0))
    return feature

In [4]:
def function(filename,stop_words):

    with open(filename, 'r',encoding='utf-8', errors='ignore') as content_file:

        content = content_file.read()
        l=[]
        l=content.split()
        
        words=[]
        punct = string.punctuation
        fake=['newsgroups', 'rob', 'strom', 'subject', 're', 'socmotss', 'et', 'al', 'princeton', 'axes', 'matching', 'funds', 'boy']
        
        
        for w in l:
            if w not in stop_words and w not in fake and len(w)>1 and len(w)<=15 and w not in punct :
                w=''.join(e for e in w if e.isalnum())
                words.append(w.lower())
        
        
            
        x = 'xref'
        while x in words:
            words.remove(x)
            
        x = 'x'
        while x in words:
            words.remove(x)
            
        x = '>>'
        while x in words:
            words.remove(x)
        
        x = "i'm"
        while x in words:
            words.remove(x)
            
        x = 'it'

        while x in words:
            words.remove(x)
            
        x = 'path'
        while x in words:
            words.remove(x)
            
        x = 'it.'
        while x in words:
            words.remove(x)
            
            
        x = 'from'
        while x in words:
            words.remove(x)
            
            
        x = '0'
        while x in words:
            words.remove(x)
            
            
        x = 'mr .'
        while x in words:
            words.remove(x)
            
            
        x = '(or'
        while x in words:
            words.remove(x)
            
        x = 'us'
        while x in words:
            words.remove(x)
            
            
        x = "i'd"
        while x in words:
            words.remove(x)
            
        x='etc.'
        while x in words:
            words.remove(x)
            
        x='(and'
        while x in words:
            words.remove(x)
            
        x='>>the'
        while x in words:
            words.remove(x)
            
        x="let's"
        while x in words:
            words.remove(x)
        
        x='l.'
        while x in words:
            words.remove(x)
            
        x='(see'
        while x in words:
            words.remove(x)
            
        x='**'
        while x in words:
            words.remove(x)
            
        x='>not'
        while x in words:
            words.remove(x)
            
            
        x='is.'
        while x in words:
            words.remove(x)
            
            
        x = '(which'
        while x in words:
            words.remove(x)
            
            
        x='>for'
        while x in words:
            words.remove(x)
            
            
        x=':)'
        while x in words:
            words.remove(x)
            
        x='(for'
        while x in words:
            words.remove(x)
            
            
        x = '|>'
        while x in words:
            words.remove(x)
            
      

        counts={}
        for p in words:
            counts[p]=(l.count(p))
    
    return counts

In [5]:
def makeVocab(subfolders):
    vocab={}
    for i in subfolders:
        for filename in glob(os.path.join(i,'*')):
            ct={}
            stop_words=(stopwords.words("english"))
        
            ct=function(filename,stop_words)
            vocab.update(ct)
    return vocab

In [6]:
def makeTrainingData(subfolders,features,istraining,path):
    output=[]
    subfolder=next(os.walk(path))[1]
    j=0
    for i in subfolders:
        for filename in glob(os.path.join(i,'*')):
                with open(filename, 'r',encoding='utf-8', errors='ignore') as content_file:
        
                    content = content_file.read()
                    l=[]
                    l=content.split()
                    temp=find_features(l,features)
                    if(istraining==True):
                        temp.append(subfolder[j])
                    output.append(temp)
        j=j+1
    return output
                
        

In [7]:
def PassDirectorynameTobuildData(subfolders,istraining,path):
    vocab=makeVocab(subfolders)
    vocab=nltk.FreqDist(vocab)

    features=list(vocab.keys())[:3000]
    #print(features)
    data=makeTrainingData(subfolders,features,istraining,path)
    if(istraining==True):
        features.append('output')
    data=np.array(data)
    df=pd.DataFrame(data)
    df.columns=features
    return df

# Naive Bayes

In [8]:
def probability(dictionary, x, current_class):
    output = 0
    features = dictionary[current_class].keys()
    for j in range(len(features) - 1):
        count_class_and_feature =  x[j]+1
        possible_values_current_feature = dictionary[current_class][j] + 3000
        
        count_class =  possible_values_current_feature
        p = np.log(count_class_and_feature) - np.log(count_class)
        output = output + p
    count_class = dictionary[current_class]['total_count']
    total_count = dictionary['total_count']
    class_prob = np.log(count_class) - np.log(total_count)
    output = output + class_prob
    return output

In [9]:
def predictSinglePoint(x, dictionary):
    classes = dictionary.keys()
    best_p = -1
    best_class = -1
    first_run = True
    for current_class in classes:
        if (current_class == 'total_count'):
            continue
        p_current_class = probability(dictionary, x, current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [10]:
def predict(dictionary, x_test):
    y_pred = []
    for x in x_test:
        x_class = predictSinglePoint(x, dictionary)
        y_pred.append(x_class)
    return y_pred

In [11]:
def fit(data):
    output_name = data.columns[-1]
    features = data.columns[0:-1]
    counts = {}
    counts["total_count"] = len(data)
    possible_outputs = set(data[output_name])
    for output in possible_outputs:
        counts[output] = {}
        smallData = data[data[output_name] == output]
        counts[output]["total_count"] = len(smallData)
        for i in range(len(features)):
            f = features[i]
            possible_values = list(map(int, smallData[features[i]]))
            counts[output][i] = sum(possible_values)
    return counts

2000


In [12]:
path='C:\\Users\\this pc\\Desktop\\SUMMER\\Cognizance_Machine_Learning\\Text_Classification\\mini_newsgroups'
subfolders = [f.path for f in os.scandir('C:\\Users\\this pc\\Desktop\\SUMMER\\Cognizance_Machine_Learning\\Text_Classification\\mini_newsgroups') if f.is_dir() ]
df=PassDirectorynameTobuildData(subfolders,True,path)


In [13]:

path='C:\\Users\\this pc\\Desktop\\SUMMER\\Cognizance_Machine_Learning\\Text_Classification\\mini_newsgroups'
subfolders = [f.path for f in os.scandir('C:\\Users\\this pc\\Desktop\\SUMMER\\Cognizance_Machine_Learning\\Text_Classification\\mini_newsgroups') if f.is_dir() ]
df1=PassDirectorynameTobuildData(subfolders,False,path)
x_test=np.array(df1)


In [14]:
dictionary = fit(df)
dictionary.keys()

dict_keys(['total_count', 'comp.os.ms-windows.misc', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'talk.politics.misc', 'misc.forsale', 'soc.religion.christian', 'sci.space', 'comp.windows.x', 'rec.sport.hockey', 'talk.politics.mideast', 'talk.politics.guns', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'sci.crypt', 'sci.med', 'talk.religion.misc', 'alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware'])

In [ ]:
dictionary['sci.space'].keys()

In [16]:
y_pred = predict(dictionary, x_test)
(y_pred==df['output'])

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1970    False
1971    False
1972    False
1973    False
1974    False
1975    False
1976    False
1977    False
1978    False
1979    False
1980    False
1981    False
1982    False
1983    False
1984    False
1985    False
1986    False
1987    False
1988    False
1989    False
1990    False
1991    False
1992    False
1993    False
1994    False
1995    False
1996    False
1997    False
1998    False
1999    False
Name: output, Length: 2000, dtype: bool

In [17]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(df['output'], y_pred)
print(ac)

0.05


# INBUILT MULTINOMIALNB

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

x = df.loc[:, df.columns != 'output']
y = df.output
x_train, x_test, y_train, y_test = train_test_split(x, y)

clf.fit(x_train, y_train)

y_pred_in = clf.predict(x_test)
#clf.score(x_test, y_test)

ac = accuracy_score(y_test, y_pred_in)
print(ac)

0.336
